In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.stem.porter import PorterStemmer
from string import punctuation
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from datetime import datetime

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/omarelshobky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/omarelshobky/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
train = pd.read_csv("Dataset/train.csv" ,encoding='unicode_escape')
train.head(5)

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


now i will drop country and population ect... because they are irrelevant to the problem, also dropping Age of user because this feature is not supported in the other dataset and also not that relevant to the problem. Also convert categorical features into features into float types.

In [3]:
train.drop(axis = 1,labels = ['Population -2020','Land Area (Km²)','Country','Age of User',"Density (P/Km²)",'text'],inplace=True)
train = train[train.selected_text.notnull()]
train.reset_index(drop=True,inplace = True )

train['Time of Tweet'][train['Time of Tweet'] == 'morning'] = 0
train['Time of Tweet'][train['Time of Tweet'] == 'noon'] = 0.5 
train['Time of Tweet'][train['Time of Tweet'] == 'night'] = 1
train['Time of Tweet'] = (train['Time of Tweet'].astype('float'))

train['sentiment'][train['sentiment'] == 'negative'] = 0
train['sentiment'][train['sentiment'] == 'neutral'] = 1
train['sentiment'][train['sentiment'] == 'positive'] = 2
train['sentiment'] = (train['sentiment'].astype('int'))

train.head(5)

,textID,selected_text,sentiment,Time of Tweet
0,cb774db0d1,"I`d have responded, if I were going",1,0.0
1,549e992a42,Sooo SAD,0,0.5
2,088c60f138,bullying me,0,1.0
3,9642c003ef,leave me alone,0,0.0
4,358bd9e861,"Sons of ****,",0,0.5


In [4]:
train2 = pd.read_csv("Dataset/training.1600000.processed.noemoticon.csv",delimiter=',', encoding='ISO-8859-1')
train2.head(5)

,polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


we will keep only the columns in the previous dataset and cahnge the column names to match

In [5]:
train2 = train2[train2[train2.columns[-1]].notnull()]
train2['sentiment'] = (train2['polarity of tweet\xa0']//2)
train2['textID'] = train2[train2.columns[1]]
train2['Time of Tweet'] = train2[train2.columns[2]]
train2.drop(labels = [train2.columns[0],train2.columns[2],train2.columns[3],train2.columns[4]],axis=1,inplace=True)

train2['selected_text'] = train2[train2.columns[1]]
train2 = train2[['textID','selected_text','sentiment','Time of Tweet']]
train2.head(5)

,textID,selected_text,sentiment,Time of Tweet
0,1467810672,is upset that he can't update his Facebook by ...,0,Mon Apr 06 22:19:49 PDT 2009
1,1467810917,@Kenichan I dived many times for the ball. Man...,0,Mon Apr 06 22:19:53 PDT 2009
2,1467811184,my whole body feels itchy and like its on fire,0,Mon Apr 06 22:19:57 PDT 2009
3,1467811193,"@nationwideclass no, it's not behaving at all....",0,Mon Apr 06 22:19:57 PDT 2009
4,1467811372,@Kwesidei not the whole crew,0,Mon Apr 06 22:20:00 PDT 2009


in this dataset time is given following a 24 hour time clock, so we need to categorize it. 0 -> 8 (8 AM) is considered night, 8 -> 16 (4 PM) is considered morning, 16 -> 0 (12 AM) is considered noon.

In [6]:
def categ_time(arr_of_dates):
    arr_of_time = np.zeros(len(arr_of_dates), dtype = float)
    i=0
    for d in arr_of_dates:
        t = datetime.strptime(d, '%a %b %d %H:%M:%S PDT %Y')
        t =  t.hour
        if (t >= 0 and t < 8):
            t = 1  #night
        elif(t >= 8 and t<= 16):
            t = 0 #morning
        else:
            t= 0.5
        arr_of_time[i] += t
        i += 1
    return arr_of_time

In [7]:
train2['Time of Tweet'] = categ_time(train2['Time of Tweet'][:].values)
train2.head(5)

,textID,selected_text,sentiment,Time of Tweet
0,1467810672,is upset that he can't update his Facebook by ...,0,0.5
1,1467810917,@Kenichan I dived many times for the ball. Man...,0,0.5
2,1467811184,my whole body feels itchy and like its on fire,0,0.5
3,1467811193,"@nationwideclass no, it's not behaving at all....",0,0.5
4,1467811372,@Kwesidei not the whole crew,0,0.5


In [8]:
train2['sentiment'].value_counts()

0    799996
2    248576
Name: sentiment, dtype: int64

In [9]:
train['sentiment'].value_counts()

1    11117
2     8582
0     7781
Name: sentiment, dtype: int64

there is a huge bias in the classes. and not enough neutral data to train. so we might stick to predicting negative and positive labels only.

In [10]:
neg = train[train['sentiment'] == 0]
pos = train[train['sentiment'] == 2]

In [11]:
neg2 = train2[train2['sentiment'] == 0]
pos2 = train2[train2['sentiment'] == 2]

In [12]:
pos = pos.append(pos2)
neg = neg.append(neg2)
neg = neg.sample(replace=False,n=len(pos),random_state=0)

In [13]:
train = pos.append(neg)
train['sentiment'].value_counts()

2    257158
0    257158
Name: sentiment, dtype: int64

In [14]:
train['sentiment'] = train['sentiment']//2

In [15]:
train = train.sample(frac = 1).reset_index(drop=True)
train

,textID,selected_text,sentiment,Time of Tweet
0,2215717472,SYTYCD this season is SO hard to judge because...,0,0.5
1,2306024945,@patrick_ritchie well don't tease us like that,0,0.5
2,1880362486,Kyla is so great and so nice i like her songs....,1,1.0
3,1835719146,@IamButton oh Hai! Are u ok after the quake a...,1,1.0
4,2069211499,I still can't believe I get a 3 week break for...,0,0.0
...,...,...,...,...
514311,1556384244,1:35am....I found my book manuscripts I wrote ...,1,0.5
514312,1694444031,"has a busy day ahead of her, doing uni work -s...",0,1.0
514313,1694708048,is working. I am so happy my job is a radio ...,1,1.0
514314,1880478917,@jtroyer I &lt;3 VMWare! it is one of the b...,1,1.0


In [16]:
train['Time of Tweet'].value_counts()

1.0    234039
0.5    147199
0.0    133078
Name: Time of Tweet, dtype: int64

this bias is acceptable. now we need to preprocess the text now.

now to the text preprocessing. i will loop over all tweets,using regular expressions ,remove words that contain more than 2 consecutive identical chatracters like (aaahhh, zzzz) will be converted to (   ,    ), and remove special characters from words leave only '*' to recognize swear words. @ to recognize a username which to be removed later. then applying pos_tag to determine the word type.then loop over every word in the tweet and its type.if word is a letter then skip .if word contains * then it is a swear and counted as one and added back to the tweet. else i only add nouns verbs adjectives and adverbs back to the tweet ,only if it was not. a link or a username then dont ad it back to the tweet. so now we have tweets as words of the allowed type no letters no usernames no links.

In [18]:
def preprocess_text(tweets):
    stemmer = PorterStemmer()
    text = [] # array of new tweets
    swears = [] # no of swears per tweet
    bag_of_words = set({})
    for i in range(0,len(tweets)):
        if(i%10000 == 0):
            print(f'iteration :{i}')
        s = 0 # swear counter
        tweet = re.sub('[^a-zA-Z*@\s\n\']', ' ', tweets[i]) # removes any none alphabetic letter + allow swears and usernames to be removed as a whole later
        tweet = re.sub(r'((\w)\2{2,})', ' ', tweet) # to remove words like aaaaah zzzzz etc.
        tweet = tweet.lower()
        tweet = tweet.split()
        val = nltk.pos_tag(tweet) # gets word type
        tweet = []
        for j in range(0,len(val)-1):
            val2 = val[j]
            stemmed_word = stemmer.stem(val2[0])
            if(len(val2[0]) <= 1):
                continue
            if("*" in stemmed_word): 
                tweet.append('*')
                bag_of_words.add('*')
                s += 1
            elif(val2[1] in ['NN','NNS','NNPS','NNP','VBZ','VBP','VBN','VBG','VBD','VB','RBS', 'RB', 'RBR','JJ','JJR','JJS']): # allowing verbs nouns adjctives and adverbs only
                if (not (('http' in stemmed_word) or ('@' in stemmed_word) or ('www' in stemmed_word))): # and also word has to be not a link or a username
                    tweet.append(stemmed_word)
                    bag_of_words.add(stemmed_word)
        
        swears.append(s)   

        tweet = [word for word in tweet]
        text.append(' '.join(tweet))
    
    return text,bag_of_words,swears

this function returns final(dataframe of [features + swear column(just for analysis not for the model]),Feat_vec(np array [words + time]),labels,bag of words just for analysis, tfidf_v)

In [19]:
def FE(train,text_col_name = 'selected_text' , max_feat = 10000, ngram = (1,1)):
    labels = train['sentiment'].values
    msgs1,bag,swears = preprocess_text(train[text_col_name])

    tfidf_v = TfidfVectorizer(max_features=max_feat,ngram_range=ngram)
    Feat_vec = tfidf_v.fit_transform(msgs1).toarray()
    cols = tfidf_v.get_feature_names_out()
    print("After vectorization")
    final = pd.DataFrame(Feat_vec,columns = cols)
    Feat_vec = np.column_stack([Feat_vec,(train['Time of Tweet'][:].values)])
    
    print("after adding")
    final['Time of Tweet'] = train['Time of Tweet'][:]
    final['number_of_swears'] = np.array(swears,dtype=float)
    
    return final,Feat_vec,labels,bag,tfidf_v

In [20]:
df,features,labels,bag,tfidf_v = FE(train)

iteration :0
iteration :10000
iteration :20000
iteration :30000
iteration :40000
iteration :50000
iteration :60000
iteration :70000
iteration :80000
iteration :90000
iteration :100000
iteration :110000
iteration :120000
iteration :130000
iteration :140000
iteration :150000
iteration :160000
iteration :170000
iteration :180000
iteration :190000
iteration :200000
iteration :210000
iteration :220000
iteration :230000
iteration :240000
iteration :250000
iteration :260000
iteration :270000
iteration :280000
iteration :290000
iteration :300000
iteration :310000
iteration :320000
iteration :330000
iteration :340000
iteration :350000
iteration :360000
iteration :370000
iteration :380000
iteration :390000
iteration :400000
iteration :410000
iteration :420000
iteration :430000
iteration :440000
iteration :450000
iteration :460000
iteration :470000
iteration :480000
iteration :490000
iteration :500000
iteration :510000
After vectorization
after adding


In [21]:
df.head(5)

,aa,aah,aaron,ab,abandon,abc,abil,abit,abl,about,...,yummi,yup,yy,zac,zealand,zombi,zone,zoo,Time of Tweet,number_of_swears
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


more analysis on swears and sentiment

In [22]:
df['number_of_swears'][labels==0].value_counts()

0.0    254979
1.0      1795
2.0       354
3.0        27
4.0         2
6.0         1
Name: number_of_swears, dtype: int64

In [23]:
df['number_of_swears'][labels==1].value_counts()

0.0    254994
1.0      1722
2.0       418
3.0        16
4.0         7
5.0         1
Name: number_of_swears, dtype: int64

In [24]:
df['number_of_swears'][labels==2].value_counts()

Series([], Name: number_of_swears, dtype: int64)

this shows that swears dont show relevancy. the distribution is similar in all categories given that we have less neutral samples.

In [25]:
"""split 0.4"""
split = int(0.4*(len(features)))
X_train = features[:split+1]
y_train = labels[:split+1]

In [26]:
X_test = features[split+1:]
y_test = labels[split+1:]

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
model=LogisticRegression(penalty = 'l2',solver='liblinear',C = 140, random_state=0)

In [29]:
model.fit(X_train,y_train)

LogisticRegression(C=140, random_state=0, solver='liblinear')

In [30]:
yp = model.predict(X_test)
training_data_accuracy = metrics.accuracy_score(yp, y_test)
print('Accuracy score of the training data:',training_data_accuracy)

Accuracy score of the training data: 0.7619552219943031


76% score with logistic regression.

In [76]:
test1 = pd.read_csv("Dataset/test.csv",encoding= 'unicode_escape')

test1.drop(axis = 1,labels = ['Population -2020','Land Area (Km²)','Country','Age of User',"Density (P/Km²)"],inplace=True)
test1 = test1[test1.text.notnull()]
test1.reset_index(drop=True,inplace = True )

test1['Time of Tweet'][test1['Time of Tweet'] == 'morning'] = -1 
test1['Time of Tweet'][test1['Time of Tweet'] == 'noon'] = 0
test1['Time of Tweet'][test1['Time of Tweet'] == 'night'] = 1
test1['Time of Tweet'] = (test1['Time of Tweet'].astype('float'))

test1 = test1[test1['sentiment'] != 'neutral']
test1['sentiment'][test1['sentiment'] == 'negative'] = 0
test1['sentiment'][test1['sentiment'] == 'positive'] = 1
test1['sentiment'] = (test1['sentiment'].astype('int'))

test1.head(5)

,textID,text,sentiment,Time of Tweet
1,96d74cb729,Shanghai is also really exciting (precisely -...,1,0.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",0,1.0
3,01082688c6,happy bday!,1,-1.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,1,0.0
5,726e501993,that`s great!! weee!! visitors!,1,1.0


In [77]:
test1['sentiment'].value_counts()

1    1103
0    1001
Name: sentiment, dtype: int64

In [78]:
yt = test1['sentiment'].values
idx = test1['textID'].values

#features
xt = tfidf_v.transform(test1['text']).todense()
xt = np.column_stack([xt,test1['Time of Tweet'].values])



ytp = model.predict(xt)

submission = pd.DataFrame({'id':idx,'label':list(ytp)})
submission.to_csv('Dataset/submit.csv')

In [79]:
submission.head(10)

,id,label
0,96d74cb729,1
1,eee518ae67,0
2,01082688c6,0
3,33987a8ee5,1
4,726e501993,1
5,261932614e,0
6,afa11da83f,0
7,24c92644a4,0
8,5c1e0b61a1,0
9,504e45d9d9,0


In [80]:
training_data_accuracy = metrics.accuracy_score(ytp,yt)
print('Accuracy score of the training data:',training_data_accuracy)

Accuracy score of the training data: 0.6863117870722434
